# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, observables: List[str] = []) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negative.
 |   

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,419.234439,-0.706892,1234,418.527546,419.941331,837.055092,839.882662
6,159.468491,-0.070702,1234,159.397789,159.539193,318.795578,319.078386
7,984.733075,0.398516,1234,985.131591,984.334558,1970.263182,1968.669117
8,6.381994,-0.929425,1234,5.452570,7.311419,10.905139,14.622839
9,809.406950,-0.740542,1234,808.666408,810.147491,1617.332816,1620.294983
10,443.854475,-0.770524,1234,443.083951,444.625000,886.167902,889.249999
11,625.340518,0.421013,1234,625.761531,624.919505,1251.523061,1249.839011
12,476.730452,-0.854194,1234,475.876258,477.584646,951.752516,955.169291
13,16.812082,0.849970,1234,17.662052,15.962112,35.324104,31.924224
14,517.464449,0.524402,1234,517.988851,516.940046,1035.977702,1033.880092


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-418.527546,419.941331
6,-159.397789,159.539193
7,-985.131591,984.334558
8,-5.452570,7.311419
9,-808.666408,810.147491
10,-443.083951,444.625000
11,-625.761531,624.919505
12,-475.876258,477.584646
13,-17.662052,15.962112
14,-517.988851,516.940046


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-418.527546,419.941331
6,-159.397789,159.539193
7,-985.131591,984.334558
8,-5.452570,7.311419
9,-808.666408,810.147491
10,-443.083951,444.625000
11,-625.761531,624.919505
12,-475.876258,477.584646
13,-17.662052,15.962112
14,-517.988851,516.940046


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-418.52754623,  419.94133095],
       [-159.39778902,  159.53919281],
       [-985.13159107,  984.33455825],
       [  -5.45256956,    7.31141942],
       [-808.66640801,  810.14749144],
       [-443.08395092,  444.62499956],
       [-625.76153066,  624.91950527],
       [-475.8762582 ,  477.58464574],
       [ -17.66205213,   15.96211193],
       [-517.98885116,  516.94004618]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,835.055092,-839.882662
6,316.795578,-319.078386
7,1968.263182,-1968.669117
8,8.905139,-14.622839
9,1615.332816,-1620.294983
10,884.167902,-889.249999
11,1249.523061,-1249.839011
12,949.752516,-955.169291
13,33.324104,-31.924224
14,1033.977702,-1033.880092


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
